In [3]:
import surprise

surprise.__version__

'1.1.1'

In [ ]:
from surprise import Dataset
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.25, random_state=0

In [ ]:
from surprise import SVD

algo = SVD()
algo.git(trainset)

In [ ]:
predictions = algo.test(testset)
